# Rename ALL crawled files by adding PREFIX to the filename

In [11]:
PREFIX = "JY"

import os

folder_path = './data/raw_data/'

for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):

        # Check if the file already starts with PREFIX
        if filename[:len(PREFIX)] == PREFIX:
            continue

        new_filename = PREFIX + filename
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))

# Load in all data and make a .tsv file

In [18]:
import os
import csv

data_folder_path = './data/raw_data/'
tsv_path = './data/dirty_data.tsv'
table = []

for filename in os.listdir(data_folder_path):
    if os.path.isfile(os.path.join(data_folder_path, filename)):
        entry = {}
        file_id = filename.split("-")[0]
        with open(data_folder_path + filename, "r") as f:
            content = f.read()
        entry = {
            "id" : file_id,
            "filename" : filename,
            "content" : content
        }
        table.append(entry)

fieldnames = table[0].keys()
with open(tsv_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, delimiter='\t', fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(table)

## Read back the data to test implementation

In [23]:
data = []

with open(tsv_path, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        data.append(row)

print(data[1]) # data[0] will be the header

['JY321', 'JY321-rstu_api_test.py', 'import pytest\nimport pytest_asyncio\n\nfrom rtsu_students_bot.rtsu import RTSUApi\n\npytest_plugins = (\'pytest_asyncio\',)\n\nTEST_DATA = {\n    "login": "your login",\n    "password": "your pass",\n}\n\n\n@pytest_asyncio.fixture()\nasync def rtsu_client():\n    """\n    Initializes client\n    :return: Prepared `RTSUApi` client\n    """\n\n    async with RTSUApi() as api:\n        yield api\n\n\n@pytest.mark.asyncio\nasync def test_rtsu_login(rtsu_client: RTSUApi):\n    """\n    Tests rtsu login\n    :param rtsu_client: A RTSU API client\n    :return:\n    """\n\n    resp = await rtsu_client.auth(TEST_DATA.get("login"), TEST_DATA.get("password"))\n\n    assert resp.token is not None\n\n\n@pytest.mark.asyncio\nasync def test_rtsu_profile_fetching(rtsu_client: RTSUApi):\n    """\n    Tests rtsu profile fetching\n    :param rtsu_client:\n    :return:\n    """\n\n    await rtsu_client.auth(TEST_DATA.get("login"), TEST_DATA.get("password"))\n\n    pro